[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-/blob/main/notebooks/01_group_manifold_visualization.ipynb) [![GitHub](https://img.shields.io/badge/GitHub-Repository-blue?logo=github)](https://github.com/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-)

# IRH v21.1: Group Manifold Visualization

**THEORETICAL FOUNDATION**: [Intrinsic_Resonance_Holography-v21.1.md](https://github.com/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-/blob/main/Intrinsic_Resonance_Holography-v21.1.md) §1.1

This notebook provides interactive visualizations of the fundamental group manifold
$G_{\text{inf}} = \text{SU}(2) \times \text{U}(1)_\phi$ and its properties.

---

## Contents

1. [Setup](#1-setup)
2. [SU(2) as the 3-sphere](#2-su2-as-the-3-sphere)
3. [U(1) Phase Circle](#3-u1-phase-circle)
4. [Product Manifold G_inf](#4-product-manifold-g_inf)
5. [Quaternion Algebra](#5-quaternion-algebra)
6. [QNCD Metric Visualization](#6-qncd-metric-visualization)

## 1. Setup

In [ ]:
# Install IRH on Colab
import sys
if 'google.colab' in sys.modules:
    print("Running on Google Colab - Installing IRH...")
    !git clone https://github.com/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-.git /content/IRH
    sys.path.insert(0, '/content/IRH')
    %cd /content/IRH
    !pip install -q numpy scipy sympy matplotlib
    print("\n✓ IRH installed successfully!")
else:
    print("Running locally")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Import IRH modules
from src.primitives.quaternions import Quaternion, verify_quaternion_algebra
from src.primitives.group_manifold import GInfElement, SU2Element

print("✓ Modules imported successfully!")

## 2. SU(2) as the 3-sphere

SU(2) is isomorphic to the 3-sphere S³, which can be parameterized by unit quaternions:

$$u = q_0 + iq_1 + jq_2 + kq_3, \quad |u|^2 = q_0^2 + q_1^2 + q_2^2 + q_3^2 = 1$$

**Reference**: Intrinsic_Resonance_Holography-v21.1.md §1.1.1

In [ ]:
# Generate random SU(2) elements (points on S³)
np.random.seed(42)
n_points = 500

# Sample from unit quaternions
def random_su2_elements(n):
    """Generate n random SU(2) elements."""
    # Generate random 4-vectors
    v = np.random.randn(n, 4)
    # Normalize to unit sphere
    norms = np.linalg.norm(v, axis=1, keepdims=True)
    return v / norms

su2_points = random_su2_elements(n_points)

# Visualize using Hopf fibration projection to S²
def hopf_projection(q):
    """Project SU(2) ≅ S³ to S² via Hopf fibration."""
    q0, q1, q2, q3 = q[:, 0], q[:, 1], q[:, 2], q[:, 3]
    x = 2 * (q1 * q3 + q0 * q2)
    y = 2 * (q2 * q3 - q0 * q1)
    z = q0**2 + q3**2 - q1**2 - q2**2
    return x, y, z

x, y, z = hopf_projection(su2_points)

# Plot
fig = plt.figure(figsize=(12, 5))

# 3D sphere projection
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(x, y, z, c=su2_points[:, 0], cmap='viridis', alpha=0.6, s=10)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.set_title('SU(2) → S² (Hopf Projection)')

# Quaternion component distribution
ax2 = fig.add_subplot(122)
ax2.hist(su2_points[:, 0], bins=30, alpha=0.7, label='$q_0$ (real)')
ax2.hist(su2_points[:, 1], bins=30, alpha=0.7, label='$q_1$ (i)')
ax2.set_xlabel('Component value')
ax2.set_ylabel('Count')
ax2.set_title('Quaternion Component Distribution')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Generated {n_points} SU(2) elements on S³")

## 3. U(1) Phase Circle

U(1)_φ represents the holonomic phase φ ∈ [0, 2π):

$$e^{i\phi} = \cos\phi + i\sin\phi$$

In [ ]:
# U(1) phase circle visualization
theta = np.linspace(0, 2*np.pi, 100)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Phase circle
ax1.plot(np.cos(theta), np.sin(theta), 'b-', linewidth=2)
# Mark some special phases
special_phases = [0, np.pi/2, np.pi, 3*np.pi/2]
labels = ['0', 'π/2', 'π', '3π/2']
for phi, label in zip(special_phases, labels):
    ax1.plot(np.cos(phi), np.sin(phi), 'ro', markersize=10)
    ax1.annotate(f'φ = {label}', (np.cos(phi)*1.15, np.sin(phi)*1.15), ha='center')
ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_aspect('equal')
ax1.set_xlabel('Re($e^{iφ}$)')
ax1.set_ylabel('Im($e^{iφ}$)')
ax1.set_title('U(1)_φ Phase Circle')
ax1.grid(True, alpha=0.3)

# Phase distribution from random sampling
random_phases = np.random.uniform(0, 2*np.pi, 1000)
ax2.hist(random_phases, bins=30, density=True, alpha=0.7)
ax2.axhline(1/(2*np.pi), color='r', linestyle='--', label='Uniform: 1/(2π)')
ax2.set_xlabel('Phase φ')
ax2.set_ylabel('Density')
ax2.set_title('U(1)_φ Haar Measure (Uniform)')
ax2.legend()

plt.tight_layout()
plt.show()

## 4. Product Manifold G_inf

The fundamental group manifold is:

$$G_{\text{inf}} = \text{SU}(2) \times \text{U}(1)_\phi$$

This 4-dimensional manifold encodes the minimal non-commutative structure (SU(2) ≅ S³) combined with holonomic phase (U(1)).

**Reference**: Intrinsic_Resonance_Holography-v21.1.md §1.1

In [ ]:
# Visualize G_inf = SU(2) × U(1)
fig = plt.figure(figsize=(14, 5))

# Generate G_inf elements
n_ginf = 200
su2_samples = random_su2_elements(n_ginf)
phi_samples = np.random.uniform(0, 2*np.pi, n_ginf)

# Project SU(2) part
x, y, z = hopf_projection(su2_samples)

# 3D plot colored by phase
ax1 = fig.add_subplot(131, projection='3d')
scatter = ax1.scatter(x, y, z, c=phi_samples, cmap='hsv', alpha=0.7, s=20)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.set_title('G_inf Elements\n(SU(2) proj., color = φ)')
plt.colorbar(scatter, ax=ax1, label='Phase φ')

# 2D projection: q0 vs phase
ax2 = fig.add_subplot(132)
ax2.scatter(su2_samples[:, 0], phi_samples, c=su2_samples[:, 1], cmap='coolwarm', alpha=0.7)
ax2.set_xlabel('$q_0$ (SU(2) real part)')
ax2.set_ylabel('Phase φ')
ax2.set_title('G_inf: SU(2) × U(1) Structure')

# Product topology diagram
ax3 = fig.add_subplot(133)
ax3.text(0.5, 0.8, '$G_{inf} = SU(2) \\times U(1)_\\phi$', fontsize=16, ha='center', transform=ax3.transAxes)
ax3.text(0.5, 0.6, '$\\cong S^3 \\times S^1$', fontsize=14, ha='center', transform=ax3.transAxes)
ax3.text(0.5, 0.4, 'dim($G_{inf}$) = 4', fontsize=12, ha='center', transform=ax3.transAxes)
ax3.text(0.5, 0.2, 'Compact, Connected', fontsize=12, ha='center', transform=ax3.transAxes)
ax3.axis('off')
ax3.set_title('Topological Properties')

plt.tight_layout()
plt.show()

## 5. Quaternion Algebra

Quaternions form a non-commutative division algebra with basis {1, i, j, k}:

$$i^2 = j^2 = k^2 = ijk = -1$$

**Reference**: Intrinsic_Resonance_Holography-v21.1.md §1.1.1

In [ ]:
# Verify quaternion algebra
results = verify_quaternion_algebra()

print("Quaternion Algebra Verification")
print("="*40)
for key, passed in results.items():
    status = "✓" if passed else "✗"
    print(f"{status} {key}: {passed}")

print(f"\nAll axioms satisfied: {results['all_passed']}")

In [ ]:
# Demonstrate non-commutativity
q1 = Quaternion(1, 2, 0, 0)
q2 = Quaternion(1, 0, 3, 0)

print("Non-commutativity demonstration:")
print(f"q1 = {q1}")
print(f"q2 = {q2}")
print(f"\nq1 × q2 = {q1 * q2}")
print(f"q2 × q1 = {q2 * q1}")
print(f"\nq1 × q2 ≠ q2 × q1: {q1 * q2 != q2 * q1}")

## 6. QNCD Metric Visualization

The Quantum Normalized Compression Distance (QNCD) provides a metric on G_inf.

**Reference**: Intrinsic_Resonance_Holography-v21.1.md Appendix A

In [ ]:
from src.primitives.qncd import compute_QNCD

# Compute QNCD distance matrix for sample elements
n_samples = 20

# Generate G_inf elements
elements = [GInfElement.random(np.random.default_rng(i)) for i in range(n_samples)]

# Compute distance matrix
distances = np.zeros((n_samples, n_samples))
for i in range(n_samples):
    for j in range(n_samples):
        if i != j:
            distances[i, j] = compute_QNCD(elements[i], elements[j])

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Distance matrix heatmap
im = ax1.imshow(distances, cmap='viridis')
ax1.set_xlabel('Element j')
ax1.set_ylabel('Element i')
ax1.set_title('QNCD Distance Matrix')
plt.colorbar(im, ax=ax1, label='QNCD(i, j)')

# Distance distribution
upper_tri = distances[np.triu_indices(n_samples, k=1)]
ax2.hist(upper_tri, bins=20, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(upper_tri), color='r', linestyle='--', label=f'Mean: {np.mean(upper_tri):.3f}')
ax2.set_xlabel('QNCD Distance')
ax2.set_ylabel('Count')
ax2.set_title('QNCD Distance Distribution')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Mean QNCD distance: {np.mean(upper_tri):.4f}")
print(f"Std QNCD distance: {np.std(upper_tri):.4f}")

---

## Summary

In this notebook, we explored:

1. **SU(2) ≅ S³**: Unit quaternions form a 3-sphere
2. **U(1)_φ**: Holonomic phase circle
3. **G_inf = SU(2) × U(1)_φ**: 4D product manifold
4. **Quaternion algebra**: Non-commutative structure
5. **QNCD metric**: Algorithmic distance on G_inf

### Next Notebooks

- [02_rg_flow_interactive.ipynb](./02_rg_flow_interactive.ipynb) - RG flow visualization
- [03_observable_extraction.ipynb](./03_observable_extraction.ipynb) - Physical constants
- [04_falsification_analysis.ipynb](./04_falsification_analysis.ipynb) - Testable predictions

### Citation

```bibtex
@software{IRH_v21_computational_2025,
  title={Intrinsic Resonance Holography v21.1: Computational Framework},
  author={McCrary, Brandon D.},
  year={2025},
  url={https://github.com/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-}
}
```